In [1]:
import keras
import sys
import matplotlib
import numpy as np
import time
import matplotlib.pyplot as plt
import os

sys.path.append('../')

from datasets import CD_Dataset
from models import Unet, MimoNet
from utility import show_batches, from_categorical, train, crop_receptive, predict_full_image
from utility import dice, precision, Pc
from datasets import combine_y_w

seed = int((time.time()*1e6)%1e6)
np.random.seed(seed)
keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=seed)

Using TensorFlow backend.


In [3]:
input_patch_size = [500,500] # input patch (expect output to be smaller)
input_channels = [3] # RGB
output_channels = [2] #cell and bkg
NBATCH = 2 # example patch per batch
EPCS = 1000 # epochs
REG = True # regularization
USEW = True # use weights
W = 10 # importance of weights
JT = True # Just train set means no evaluation during training 

trained_models_path = '../trained_models'
print('\nInput name of net: append type e.g. MIMO_expertiment_name\n')
NAME_NET = raw_input()
if 'MIMO' in NAME_NET :
    MIMO = False
    UNET = True
if 'UNET' in NAME_NET :
    MIMO = True
    UNET = False

NAME_NET = os.path.join(trained_models_path,NAME_NET)
if output_channels[0] == 3:
    train_y_path = "train_yc"
    eval_y_path = "eval_yc"
if output_channels[0] == 2:
    train_y_path = "train_y"
    eval_y_path = "eval_y"


Input name of net: append type e.g. MIMO_expertiment_name

MIMO_TEST


In [4]:
# Load the dataset
dataset_path = '../CD_Dataset'
dataset = CD_Dataset( path=dataset_path, 
                     train_y_path=train_y_path,  
                     eval_y_path=eval_y_path, 
                     fit=True, download=True, 
                     num_classes=output_channels[0] )

In [5]:
# define your model
model_input_size = input_patch_size + input_channels
if MIMO:
    model = MimoNet(model_input_size, classes=output_channels[0], regularized=REG)
if UNET:
    model = Unet(model_input_size, classes=output_channels[0], regularized=REG)
model_output_size = list(model.outputs_shape[0])
print("input size: {}\noutput_size: {}".format(model_input_size,model_output_size))

input size: [500, 500, 3]
output_size: [308, 308, 2]


In [ ]:
def train(model,dataset, epochs=10, n_batch=10, 
          use_weights=False, W=20, just_train=True, rotate=False, 
          name=None,
          train_metrics=[],eval_metrics=[] ):

    h,w,_ = model.inputs_shape[0]
    means = dataset.mean_features()
    stds = dataset.std_features()
    for i in range(epochs):
        print("=========== iteration {}/{} =============".format(i+1,epochs))
        if (i+1)%10 == 0 and not(name is None):
            print('saving model: {}'.format(name))
            model.save_model(name)
            y_hat = model.predict(x_train)
            xs_c = crop_receptive(x_train,model.outputs_shape[0][:2])
            ys_c = crop_receptive(y_train,model.outputs_shape[0][:2])
            ys_imgs_c = from_categorical(ys_c)
            y_hat_imgs = from_categorical(y_hat)
            show_batches([xs_c*stds+means,ys_imgs_c,y_hat_imgs],["xs","ys","yhat"])

        x_train,y_train,w_train = dataset.sample_X_Y_W_patch_batch([h,w],n_batch=n_batch,rotate=rotate)
        if use_weights:
            y_train = combine_y_w(y_train,w_train*W)    
        train_history = model.fit(x_train,y_train)
        train_metric = train_history.history.values()
        
        if not just_train:
            x_eval1,y_eval1,w_eval1 = dataset.sample_X_Y_W_patch_batch([h,w],n_batch=n_batch,train=False)
            x_eval2,y_eval2,w_eval2 = dataset.sample_X_Y_W_patch_batch([h,w],n_batch=n_batch,train=False)

            x_eval = np.concatenate( (x_eval1,x_eval2), axis=0 )
            y_eval = np.concatenate( (y_eval1,y_eval2), axis=0 )
            w_eval = np.concatenate( (w_eval1,w_eval2), axis=0 )

            eval_history = model.evaluate(x_eval,y_eval)
            eval_metric = eval_history

            eval_metrics += [eval_metric]
        train_metrics += [train_metric]
    if not(name is None):
        print('saving model: {}'.format(name))
        model.save_model(name)
        eval_histo = np.array(eval_metrics)
        train_histo = np.array(train_metrics)
        train_histo.dump(name+'_train_histo.pkl')
        eval_histo.dump(name+'_eval_histo.pkl')
    return eval_metrics, train_metrics

histo = train(model,dataset,n_batch=NBATCH,epochs=EPCS,just_train=JT,use_weights=USEW, W=W, name=NAME_NET)

=========== iteration 1/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 555ms/step - loss: 53.1750 - softmax_categorical_crossentropy: 0.3878 - dice_coef: 0.6990
=========== iteration 2/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 495ms/step - loss: 53.1065 - softmax_categorical_crossentropy: 0.4449 - dice_coef: 0.6894
=========== iteration 3/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 488ms/step - loss: 52.9909 - softmax_categorical_crossentropy: 0.4547 - dice_coef: 0.6822
=========== iteration 4/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 481ms/step - loss: 52.8331 - softmax_categorical_crossentropy: 0.4218 - dice_coef: 0.7076
=========== iteration 5/1000 =============
Epoch 1/1
2/2 [==============================] - 1s 484ms/step - loss: 52.6647 - softmax_categorical_crossentropy: 0.3781 - dice_coef: 0.7116
=========== iteration 6/1000 =============
Epoch 1/1
2/2 [=================